<a href="https://colab.research.google.com/github/VinitVpANDEY/Neural-Network/blob/main/AutoGrad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math

class Value:
  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data})"

  def __add__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')
    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward
    return out

  def __mul__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data * other.data, (self, other), '*')
    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad
    out._backward = _backward
    return out

  def __rmul__(self, other):
    return self * other

  def tanh(self):
    x = self.data
    t = (math.exp(2 * x) - 1) / (math.exp(2 * x) + 1)
    out = Value(t, (self,), 'tanh')
    def _backward():
      self.grad += (1 - t**2) * out.grad
    out._backward = _backward
    return out

  def exp(self):
    x = self.data
    out = Value(math.exp(x), (self,), 'exp')
    def _backward():
      self.grad += out.data * out.grad
    out._backward = _backward
    return out

  def __pow__(self, other):
    assert isinstance(other, (int, float))
    out = Value(self.data**other, (self,), f'**{other}')
    def _backward():
      self.grad += other * self.data**(other - 1) * out.grad
    out._backward = _backward
    return out

  def __truediv__(self, other):
    return self * other**-1

  def __neg__(self):
    return -self

  def __sub__(self, other):
    return self + (-other)

  def backward(self):
    topo = []
    visited = set()

    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)

    build_topo(self)
    self.grad = 1.0

    for node in reversed(topo):
      node._backward()

In [ ]:
x1 = Value(2.0, label='x1')
x2 = Value(0.0, label='x2')
w1 = Value(-3.0, label='w1')
w2 = Value(1.0, label='w2')
b = Value(6.8813735870195432, label='b')

n = x1*w1 + x2*w2 + b
o = n.tanh()
print(o)
o.backward()
print('x1', x1.grad)
print('x2', x2.grad)
print('w1', w1.grad)
print('w2', w2.grad)
print('b', b.grad)

Value(data=0.7071067811865476)
x1 -1.4999999999999996
x2 0.4999999999999999
w1 0.9999999999999998
w2 0.0
b 0.4999999999999999


In [ ]:
import random

class Neuron:
  def __init__(self, nin):  # nin -> number of inputs to each neuron
    self.w = [Value(random.uniform(-1, 1)) for _ in range(nin)]
    self.b = Value(random.uniform(-1, 1))

  def __call__(self, x):
    # Convert input x to a list of Value objects
    x_values = [Value(xi) if not isinstance(xi, Value) else xi for xi in x]
    act = sum((wi * xi for wi, xi in zip(self.w, x_values)), Value(0.0)) + self.b
    out = act.tanh()
    return out

  def paremeters(self):
    return self.w + [self.b]

class Layer:
  def __init__(self, nin, nout):
    # nin -> number of inputs to each neuron
    # nout -> number of neurons in each layer
    self.neurons = [Neuron(nin) for _ in range(nout)]

  def __call__(self, x):
    outs = [n(x) for n in self.neurons]
    return outs[0] if len(outs) == 1 else outs

  def paremeters(self):
    return [p for neuron in self.neurons for p in neuron.paremeters()]

class MLP:
  def __init__(self, nin, nouts):
    # nin -> number of inputs to each neuron
    # nouts -> array of number of neurons in each layer
    sz = [nin] + nouts
    self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]

  def __call__(self, x):
    for layer in self.layers:
      x = layer(x)                # output of one layer is input to next layer
    return x

  def paremeters(self):
    return [p for layer in self.layers for p in layer.paremeters()]

In [ ]:
x = [2.0, 3.0, -1.0]
n = MLP(3, [4, 4, 1])
n(x)

Value(data=0.16655627073622567)

In [ ]:
def train(repeats, model, xs, ygt, lr=0.01):
  for k in range(repeats):
    # forward pass
    ypred = [model(x) for x in xs]
    loss = sum([(yout - ygt)**2 for ygt, yout in zip(ygt, ypred)], Value(0.0))
    print(k, loss.data)

    # backward pass
    for p in model.paremeters():
      p.grad = 0.0

    loss.backward()

    # update
    for p in model.paremeters():
      p.data += -lr * p.grad

  return ypred


In [ ]:
n = MLP(3, [4, 4, 1])
xs = [[2.0, 3.0, -1.0], [3.0, -1.0, 0.5], [0.5, 1.0, 1.0], [1.0, 1.0, -1.0]]
ys = [1.0, -1.0, -1.0, 1.0]

ypred = train(20, n, xs, ys, lr=0.1)
print(ypred)



0 2.841363499812927
1 0.931038669687184
2 0.3513659177338814
3 0.07806042887369305
4 0.06494876326934884
5 0.0554429440487391
6 0.04825407184295991
7 0.04263897903960801
8 0.038139653729371045
9 0.03445884654216189
10 0.031395478939877625
11 0.02880881860296594
12 0.026597521904411413
13 0.024686820359555366
14 0.02302038924509224
15 0.02155502047809253
16 0.020257035014828517
17 0.019099807685749107
18 0.018062022716420524
19 0.01712642067636518
[Value(data=0.96111019135503), Value(data=-0.9589184150369641), Value(data=-0.9068822224247824), Value(data=0.9275059565460723)]
